[View in Colaboratory](https://colab.research.google.com/github/aunz/ds/blob/master/Aesop's_Fables.ipynb)

# Aesop's Fables
<!-- ![Aesop's Fables](https://images-na.ssl-images-amazon.com/images/I/51DM5uYLhZL._SX258_BO1,204,203,200_.jpg) -->


## Data
The original text can be found at http://classics.mit.edu/Aesop/fab.mb.txt




In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# get the data
data = __import__('requests').get('https://raw.githubusercontent.com/aunz/ds/master/.data/Aesops_Fables.txt').text[:-50]
print('Corpus length', len(data))

Corpus length 188540


In [3]:
# extract the data into sentences and next_chars
sentences = [] # store the extracted sequences
next_chars = [] # store the target (the follow-up characters)
max_len = 60 # extract sequences of 60 characters
step = 2 # sample a new sequence every two characters

for i in range(0, len(data) - max_len, step):
    sentences.append(data[i: i + max_len])
    next_chars.append(data[i + max_len])

print('Number of sequences:', len(sentences))

Number of sequences: 94240


In [4]:
chars = sorted(list(set(data))) # list of unique char in the corpus
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars) # turn the list into dict to map unique chars to their index in the list "chars"
print('Vectorization...')
x = np.zeros((len(sentences), max_len, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences): # one hot encode into binary arrays
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1

Unique characters: 64
Vectorization...


In [5]:
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(128, input_shape=(max_len, len(chars)), dropout=0.2))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(lr=0.01))
model.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               98816     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
Total params: 107,072
Trainable params: 107,072
Non-trainable params: 0
_________________________________________________________________


In [0]:
# function to sample the next char given the model's prediction
def sample(preds, temperature=0.5):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    preds = np.exp(preds)
    preds = preds / np.sum(preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
%%time
import time

start_time = time.time()
for epoch in range(1, 60): # train the model for 60 epochs
    start_time_inner = time.time()
    print('Epoch', epoch, end=' ')
    model.fit(x, y, batch_size=128, epochs=1) # fit the model for 1 iteration on the data
    start_index = np.random.randint(0, len(data) - max_len - 1) # select a text seed at random
    text_seed = data[start_index: start_index + max_len]
    print('--- Generating with seed: "' + text_seed + '"')
    for temperature in [0.2, 0.5, 0.9]:
        print('------ temp:', temperature, end='. ')
        generated_text = text_seed
        print(generated_text, end='')
        for i in range(420): # generate 420 chars from the seed text
            sampled = np.zeros((1, max_len, len(chars))) # one hot encodes the chars generated so far
            for t, char in enumerate(generated_text): sampled[0, t, char_indices[char]] = 1.
            preds = model.predict(sampled, verbose=0)[0] # sample the next char
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            generated_text += next_char
            generated_text = generated_text[1:]
            print(next_char, end='')
        print() # just a line break
    print('Elapsed. This loop:', time.time() - start_time_inner, '. Total:', time.time() - start_time, '\n')
    

Epoch 1 Epoch 1/1
94240/94240 [==============================] - 110s 1ms/step - loss: 2.1772
--- Generating with seed: "sovereign. He then gave them an Eel to govern them. When the"
------ temp: 0.2. sovereign. He then gave them an Eel to govern them. When the same and the same and the sond and comperd and the same and the same and the same and the sond, and a same and said, "I his same and the wass and son and come and the somse and said, "I his fold but the same and could the sold be the sond and the sond and the sond said, "I his sond and said, "I his same and the solf said, "I his same and the sonse, and the somse for the somse, and said, "I his forned and the same fo
------ temp: 0.5. sovereign. He then gave them an Eel to govern them. When the Monter for and said to a mased the Swase and cold, and the would not and sapded, the samp for she but of the once, said, "I doge dase compers in the semped and case the hood, and campens and sald the Wolss, he have and the have bo the sors

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


 to a tree, his see of his dry complaintith to the Fox didons to pain, and cated to cholw of helpertance will not bro
Elapsed. This loop: 156.7727563381195 . Total: 3304.352104663849 

Epoch 22 Epoch 1/1
94240/94240 [==============================] - 107s 1ms/step - loss: 1.4270
--- Generating with seed: "llowed it about, so that the Partridge became grievously tro"
------ temp: 0.2. llowed it about, so that the Partridge became grievously trouble to the speection to the spes and be the flow of the forest to the streat of the same of the forest and said the Fox said the Wolf, was said the Fox said the Fox said the Fox caused the speat, who had been beasts to the forest and said the Fox came to the speection to his speece of the clot; and the traveler with the sheep with the trief of the speect of the tree the barger and said the Fleastat of the streat of 
------ temp: 0.5. llowed it about, so that the Partridge became grievously trought the willers and fled in the famier of the serpost